# 合并——笔记部分

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


## 一、append与assign

### 1.append方法

a)利用序列添加行(必须指定name)

In [4]:
df_append=df.loc[:3,['Gender','Height']].copy()
df_append

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167


In [5]:
s = pd.Series({'Gender':'F','Height':188},name='new_row')
df_append.append(s)

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_row,F,188


b)用DataFrame添加表

In [6]:

df_temp = pd.DataFrame({'Gender':['F','M'],'Height':[188,176]},index=['new_1','new_2'])
df_append.append(df_temp)

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_1,F,188
new_2,M,176


### 2.assign方法

该方法主要用于添加列，列名直接由参数指定：

In [8]:
s = pd.Series(list('abcd'),index=range(4))
df_append.assign(Letter=s)

,Gender,Height,Letter
0,M,173,a
1,F,192,b
2,M,186,c
3,F,167,d


可以一次添加多个列：

In [9]:
#用匿名函数也可以添加
df_append.assign(col1=lambda x:x['Gender']*2,col2=s)

,Gender,Height,col1,col2
0,M,173,MM,a
1,F,192,FF,b
2,M,186,MM,c
3,F,167,FF,d


## 二. combine与update

### 1.combine方法

comine和update都是用于表的填充函数，可以根据某种规则填充

a）填充对象

可以看出combine方法是按照表的顺序轮流进行逐列循环的，而且自动索引对齐，缺失值为NaN，理解这一点很重要

In [13]:
df_combine_1 = df.loc[:1,['Gender','Height']].copy()
df_combine_2 = df.loc[10:11,['Gender','Height']].copy()
df_combine_1.combine(df_combine_2,lambda x,y:print(x,y))

0       M
1       F
10    NaN
11    NaN
Name: Gender, dtype: object 0     NaN
1     NaN
10      M
11      F
Name: Gender, dtype: object
0     173.0
1     192.0
10      NaN
11      NaN
Name: Height, dtype: float64 0       NaN
1       NaN
10    161.0
11    175.0
Name: Height, dtype: float64


,Gender,Height
0,NaN,NaN
1,NaN,NaN
10,NaN,NaN
11,NaN,NaN


In [14]:
df_combine_1 = df.loc[:1,['Gender','Height']].copy()
df_combine_1

,Gender,Height
0,M,173
1,F,192


In [15]:
# 有点不太能理解，里面函数的作用，为何最后全是NaN
# combine是做并集

b)一些例子

In [24]:
# 例子1:根据列均值的大小填充
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [8, 7], 'B': [3, 4]})
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

,A,B
0,8,3
1,7,4


In [25]:
#例子2:索引对齐特性（默认状态下，后面的表（df2）没有的行列都会设置为NaN）
df2 = pd.DataFrame({'B':[8,7],'C':[6,5]},index=[1,2])

In [26]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

,A,B,C
0,NaN,NaN,NaN
1,NaN,8.0,6.0
2,NaN,7.0,5.0


In [29]:
# 这边不明白为什么A不保留1，2,对这个overwrite不是很理解
#overwrite bool, default True
#If True, columns in self that do not exist in other will be overwritten with NaNs.
#如果overwrite为true,在实例中(df1)那些没有出现在另外一个DF(df2)的列会被覆盖成空值

In [28]:
#例子3: 使得df1原来符合条件的值不会被覆盖
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y, 
            overwrite = False)

,A,B,C
0,1.0,NaN,NaN
1,2.0,8.0,6.0
2,NaN,7.0,5.0


In [30]:
#例子4:在新增匹配df2的元素位置填充-1
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=-1)

,A,B,C
0,1.0,-1.0,-1.0
1,2.0,8.0,6.0
2,-1.0,7.0,5.0


c)combin_first方法

这个方法作用是用df2填补df1的缺失值，功能比较简单，但很多时候会比combine更常用，下面举两个例子：

In [31]:
df1=pd.DataFrame({'A':[None,0],'B':[None,4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,0.0,4.0


In [32]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
df1.combine_first(df2)

,A,B,C
0,NaN,4.0,NaN
1,0.0,3.0,1.0
2,NaN,3.0,1.0


### 2.update方法

a)三个特点

1）返回的框索引只会与被调用框的一致（默认使用左连接，下一节会介绍）

2）第二个框中的nan元素不会起作用

3）没有返回值，直接在df上操作

b）例子

In [35]:
#例子1: 索引完全对齐情况下的操作
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df3=df1.update(df2)
df1

,A,B
0,1,4
1,2,5
2,3,6


In [37]:
df3#没有返回值

In [38]:
# 例子2:部分填充
df1 = pd.DataFrame({'A': ['a', 'b', 'c'],
                    'B': ['x', 'y', 'z']})
df2 = pd.DataFrame({'B': ['d', 'e']}, index=[1,2])
df1.update(df2)
df1

,A,B
0,a,x
1,b,d
2,c,e


In [39]:
# 例子3: 缺失值不会填充
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, np.nan, 6]})
df1.update(df2)
df1

,A,B
0,1,4.0
1,2,500.0
2,3,6.0


## concat方法

concat方法可以在两个维度上拼接，默认纵向凭借（axis=0），拼接方式默认外连接

所谓外连接，就是取拼接方向的并集，而'inner'时取拼接方向（若使用默认的纵向拼接，则为列的交集）的交集

下面举一些例子说明其参数：

In [40]:
df1 = pd.DataFrame({'A': ['A0', 'A1'],
                    'B': ['B0', 'B1']},
                    index = [0,1])
df2 = pd.DataFrame({'A': ['A2', 'A3'],
                    'B': ['B2', 'B3']},
                    index = [2,3])
df3 = pd.DataFrame({'A': ['A1', 'A3'],
                    'D': ['D1', 'D3'],
                    'E': ['E1', 'E3']},
                    index = [1,3])

In [42]:
df1

,A,B
0,A0,B0
1,A1,B1


In [43]:
df2

,A,B
2,A2,B2
3,A3,B3


In [44]:
df3

,A,D,E
1,A1,D1,E1
3,A3,D3,E3


In [41]:
# 默认状态拼接
pd.concat([df1,df2])

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [45]:
# axis=1时沿列方向拼接：
pd.concat([df1,df2],axis=1)

,A,B,A,B
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,NaN,NaN,A2,B2
3,NaN,NaN,A3,B3


In [46]:
# join设置为内连接（由于axis=0，因此列取交集）：
pd.concat([df3,df1],join='inner')

,A
1,A1
3,A3
0,A0
1,A1


In [49]:
pd.concat([df3,df1],axis=1,join='inner')

,A,D,E,A,B
1,A1,D1,E1,A1,B1


In [47]:
# join设置为外连接 并？
pd.concat([df3,df1],join='outer',sort=True) #sort设置列排序，默认为False

,A,B,D,E
1,A1,NaN,D1,E1
3,A3,NaN,D3,E3
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN


In [48]:
pd.concat([df3,df1],join='outer')

,A,D,E,B
1,A1,D1,E1,NaN
3,A3,D3,E3,NaN
0,A0,NaN,NaN,B0
1,A1,NaN,NaN,B1


verify_integrity检查列是否唯一

In [50]:
#pd.concat([df3,df1],verify_integrity=True,sort=True) 报错

同样，可以添加Series：

In [52]:
s = pd.Series(['X0', 'X1'], name='X')
pd.concat([df1,s],axis=0)

,A,B,0
0,A0,B0,NaN
1,A1,B1,NaN
0,NaN,NaN,X0
1,NaN,NaN,X1


In [53]:
s = pd.Series(['X0', 'X1'], name='X')
pd.concat([df1,s],axis=1)

,A,B,X
0,A0,B0,X0
1,A1,B1,X1


key参数用于对不同的数据框增加一个标号，便于索引：

In [54]:
pd.concat([df1,df2], keys=['x', 'y'])
pd.concat([df1,df2], keys=['x', 'y']).index

MultiIndex([('x', 0),
            ('x', 1),
            ('y', 2),
            ('y', 3)],
           )

In [55]:
pd.concat([df1,df2], keys=['x', 'y'])

A   B
x 0  A0  B0
  1  A1  B1
y 2  A2  B2
  3  A3  B3

## 四. merge与join

### 1.merge函数

merge函数的作用是将两个pandas对象横向合并，遇到重复的索引项时会使用笛卡尔积，默认inner连接，可选left、outer、right连接

所谓左连接，就是指以第一个表索引为基准，右边的表中如果不再左边的则不加入，如果在左边的就以笛卡尔积的方式加入

merge/join与concat的不同之处在于on参数，可以指定某一个对象为key来进行连接

感觉用数据库的连接操作类比会更好理解

In [65]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0', 'B1', 'B2', 'B3']}) 
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
right2 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3']})

In [66]:
left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [67]:
right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


In [68]:
pd.merge(left,right) # 这个应该是自然连接

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [61]:
# 以key1为准则连接，如果具有相同的列，则默认suffixes=('_x','_y')：
pd.merge(left, right, on='key1')

,key1,key2_x,A,B,key2_y,C,D
0,K0,K0,A0,B0,K0,C0,D0
1,K0,K1,A1,B1,K0,C0,D0
2,K1,K0,A2,B2,K0,C1,D1
3,K1,K0,A2,B2,K0,C2,D2
4,K2,K1,A3,B3,K0,C3,D3


以多组键连接：

In [69]:
pd.merge(left, right, on=['key1','key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


默认使用inner连接，因为merge只能横向拼接，所以取行向上keys的交集，下面看如果使用how=outer参数

In [72]:
#注意：这里的how就是concat的join 
#外连接
pd.merge(left, right, how='outer', on=['key1','key2']) 

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


In [73]:
#左连接
pd.merge(left, right, how='left', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


In [74]:
#右连接：
pd.merge(left, right, how='right', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


validate检验的是到底哪一边出现了重复索引，如果是“one_to_one”则两侧索引都是唯一，如果"one_to_many"则左侧唯一

In [75]:
left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 3, 4]})

In [76]:
pd.merge(left, right, on='B', how='outer',validate='one_to_one')

MergeError: Merge keys are not unique in left dataset; not a one-to-one merge

In [77]:
left = pd.DataFrame({'A': [1, 2], 'B': [2, 1]})
pd.merge(left, right, on='B', how='outer',validate='one_to_one')


,A_x,B,A_y
0,1.0,2,4.0
1,2.0,1,NaN
2,NaN,3,5.0
3,NaN,4,6.0


In [78]:
# indicator参数指示了，合并后该行索引的来源
df1 = pd.DataFrame({'col1': [0, 1], 'col_left': ['a', 'b']})
df2 = pd.DataFrame({'col1': [1, 2, 2], 'col_right': [2, 2, 2]})
pd.merge(df1, df2, on='col1', how='outer', indicator=True) #indicator='indicator_column'也是可以的


,col1,col_left,col_right,_merge
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


### 2. join函数

In [79]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K1', 'K2'])
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                    index=['K0', 'K2', 'K3'])
left.join(right) #默认左连接

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


In [80]:
left

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2


In [81]:
right

,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [85]:
#对于many_to_one模式下的合并，往往join更为方便
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key': ['K0', 'K1', 'K0', 'K1']})
right = pd.DataFrame({'C': ['C0', 'C1'],
                      'D': ['D0', 'D1']},
                     index=['K0', 'K1'])
left.join(right, on='key')

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K0,C0,D0
3,A3,B3,K1,C1,D1


In [83]:
left

,A,B,key
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K0
3,A3,B3,K1


In [84]:
right

,C,D
K0,C0,D0
K1,C1,D1


In [86]:
# 多层Key
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1']})
index = pd.MultiIndex.from_tuples([('K0', 'K0'), ('K1', 'K0'),
                                   ('K2', 'K0'), ('K2', 'K1')],names=['key1','key2'])
right = pd.DataFrame({'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']},
                     index=index)
left.join(right, on=['key1','key2'])

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A3,B3,K2,K1,C3,D3


In [87]:
left

,A,B,key1,key2
0,A0,B0,K0,K0
1,A1,B1,K0,K1
2,A2,B2,K1,K0
3,A3,B3,K2,K1


In [88]:
right

C   D
key1 key2        
K0   K0    C0  D0
K1   K0    C1  D1
K2   K0    C2  D2
     K1    C3  D3

# 问题

1. 请思考什么是append/assign/combine/update/concat/merge/join各自最适合使用的场景，并举出相应的例子。

append用于添加行

assign用于添加列

combine用于将两个表合为一张表，并根据所定的规则合并

update用于更新，将新的表更新到旧的表

concat用于拼接两张表

merge类似与数据库里的连接操作，但是默认为内连接操作

join也类似连接，默认为左外连接

2.merge_ordered和merge_asof的作用是什么？和merge是什么关系？

merge_ordered:

Perform merge with optional filling/interpolation.

Designed for ordered data like time series data. Optionally perform group-wise merge (see examples).

没太看懂官网的例子

# 练习

In [118]:
df1=pd.read_csv('data/Employee1.csv')
df1.head()

,Company,Name,Age,Height,Weight,Salary
0,A,a1,47,188,63.7,25819
1,A,a3,39,172,55.9,21983
2,A,a4,43,158,62.5,21755
3,A,a6,42,182,76.9,17354
4,A,a7,49,171,94.6,6177


In [95]:
df2=pd.read_csv('data/Employee2.csv')
df2.head()

,Company,Name,Age,Height,Weight,Salary
0,A,a1,30,156,91.2,28133
1,A,a2,50,190,83.4,6673
2,A,a3,34,168,96.6,16503
3,A,a5,51,176,97.2,23294
4,A,a6,37,183,93.2,19256


a)每个公司有多少员工满足如下条件：既出现第一张表，又出现在第二张表。

In [100]:
pd.merge(df1,df2,on='Name').shape[0]

16

b)将所有不符合(a)中条件的行筛选出来，合并为一张新表，列名与原表一致。

In [105]:
a =  pd.merge(df1,df2,on='Name')['Name']
b = df1.append(df2)

In [115]:
result = b.where(b['Name'].isin(a)).dropna()
result

,Company,Name,Age,Height,Weight,Salary
0,A,a1,47.0,188.0,63.7,25819.0
1,A,a3,39.0,172.0,55.9,21983.0
3,A,a6,42.0,182.0,76.9,17354.0
10,B,b1,49.0,169.0,95.9,29223.0
11,B,b3,33.0,185.0,59.7,5126.0
14,B,b7,43.0,167.0,66.5,26546.0
18,B,b15,50.0,174.0,72.2,14685.0
19,C,c3,37.0,174.0,82.8,22429.0
24,C,c10,38.0,162.0,82.8,20045.0
25,C,c12,42.0,173.0,57.4,4319.0


In [112]:
b['Name'].isin(a)

0      True
1      True
2     False
3      True
4     False
      ...  
32    False
33     True
34     True
35     True
36    False
Name: Name, Length: 77, dtype: bool

c)现在需要编制所有80位员工的信息表，对于(b)中的员工要求不变，对于满足(a)条件员工，它们在某个指标的数值，取偏离它所属公司中满足(b)员工的均值数较小的哪一个，例如：P公司在两张表的交集为{p1}，并集扣除交集为{p2,p3,p4}，那么如果后者集合的工资均值为1万元，且p1在表1的工资为13000元，在表2的工资为9000元，那么应该最后取9000元作为p1的工资，最后对于没有信息的员工，利用缺失值填充。